# 기본 설정

### 참고할 만한 사이트들
- Recommender using NLP: https://medium.com/@armandj.olivares/building-nlp-content-based-recommender-systems-b104a709c042  
- TfidfVectorizer 정리된 곳: https://chan-lab.tistory.com/27


In [5]:
from collections import Counter
import konlpy
from konlpy.tag import Mecab
from konlpy.tag import Okt

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import fasttext
from gensim.models import Word2Vec, FastText
from rake_nltk import Rake

In [2]:
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)

# petr.eat 데이터 일루와!!!

In [36]:
df = pd.read_excel('제품별 원재료 및 성분표.xlsx')

In [37]:
temp = df.copy()

In [38]:
temp.head(2)

,상품명,제조사,딱딱한 정도,주요성분,영양소,어디에 좋냐,효능,특징
0,한우사골육수,바르다펫,액체,"우족, 사골뼈, 황태, 표고버섯,대추","콜라겐, 콘드로이틴","피부건강, 관절",기력회복,기호성 좋아 특식과 함께 곁들여 먹여라
1,견냥갱(장어),바르다펫,푸딩,"바다장어, 황태, 한천가루","불포화지방산, 오메가3, 철분과 칼슘 비타민A, 필수지방산 DHA, EPA, 바다장...","혈액순환, 성장발육, 뇌건강, 슬개골탈구예방, 관절건강",NaN,NaN


In [39]:
temp.columns

Index(['상품명', '제조사', '딱딱한 정도', '주요성분 ', '영양소 ', '어디에 좋냐', '효능', '특징'], dtype='object')

In [40]:
temp['주요성분 '] = temp['주요성분 '].str.replace(',', ' ')
temp['영양소 '] = temp['영양소 '].str.replace(',', ' ')
temp['어디에 좋냐'] = temp['어디에 좋냐'].str.replace(',', ' ')
temp['효능'] = temp['효능'].str.replace(',', ' ')

In [50]:
temp = temp.fillna('')

In [51]:
temp['BOW'] = (temp['주요성분 '].astype(str) + ' ' 
               + temp['영양소 '].astype(str) + ' ' 
               + temp['어디에 좋냐'].astype(str) + ' ' 
               + temp['효능'].astype(str))

In [52]:
temp

,상품명,제조사,딱딱한 정도,주요성분,영양소,어디에 좋냐,효능,특징,BOW
0,한우사골육수,바르다펫,액체,우족 사골뼈 황태 표고버섯 대추,콜라겐 콘드로이틴,피부건강 관절,기력회복,기호성 좋아 특식과 함께 곁들여 먹여라,우족 사골뼈 황태 표고버섯 대추 콜라겐 콘드로이틴 피부건강 관절 기력회복
1,견냥갱(장어),바르다펫,푸딩,바다장어 황태 한천가루,불포화지방산 오메가3 철분과 칼슘 비타민A 필수지방산 DHA EPA 바다장...,혈액순환 성장발육 뇌건강 슬개골탈구예방 관절건강,,,바다장어 황태 한천가루 불포화지방산 오메가3 철분과 칼슘 비타민A 필수지방...
2,치킨사골육수,바르다펫,액체,닭발 닭가슴살 황태 표고버섯 대추,콜라겐 콘드로이틴,피부건강 관절건강,기력회복,,닭발 닭가슴살 황태 표고버섯 대추 콜라겐 콘드로이틴 피부건강 관절건강 ...
3,스팀제주말고기소시지,바르다펫,부드러움,말고기 당근 케일 슈퍼푸드 마키베리 표고버섯 아기치즈,팔미톨레산 베타카로틴 베타카로틴 칼슘 비타민 안토시아닌 비타민 무기질 ...,췌장 인슐린 당뇨 항산화작용 활성산소배출 노화방지 눈건강 노화방지 ...,,,말고기 당근 케일 슈퍼푸드 마키베리 표고버섯 아기치즈 팔미톨레산 베타...
4,강아지 보양식 바른 삼계탕,바르다펫,부드러움,닭발 무항생제 닭안심 소떡심 당근 브로컬리 표고버섯 황태 대추 달걀 ...,콜라겐 콘드로이틴,피부 관절건강,,,닭발 무항생제 닭안심 소떡심 당근 브로컬리 표고버섯 황태 대추 달걀 ...
5,한입오리,바르다펫,촉촉한 육포,오리안심,단백질 불포화지방산,피부 혈관건강,,,오리안심 단백질 불포화지방산 피부 혈관건강
6,한입치킨,바르다펫,촉촉한 육포,닭안심,단백질 니아신,근육강화 신진대사 면역력,,,닭안심 단백질 니아신 근육강화 신진대사 면역력
7,무염황태체,바르다펫,바삭바삭,황태,아미노산 메티오닌,간기능,,,황태 아미노산 메티오닌 간기능
8,오리목뼈,바르다펫,딱딱,오리목뼈,단백질 무기질,콜레스테롤저하 독소배출,스트레스해소 치석제거,,오리목뼈 단백질 무기질 콜레스테롤저하 독소배출 스트레스해소 치석제거
9,홍연어저키,바르다펫,마른육포,홍연어,DHA EPA 오메가3 저지방산,향산화효과 시력개선 심혈관개선 면역력강화 항암효과,,,홍연어 DHA EPA 오메가3 저지방산 향산화효과 시력개선 심혈관개선 ...


In [53]:
df = temp[['상품명', 'BOW']]

In [54]:
df

,상품명,BOW
0,한우사골육수,우족 사골뼈 황태 표고버섯 대추 콜라겐 콘드로이틴 피부건강 관절 기력회복
1,견냥갱(장어),바다장어 황태 한천가루 불포화지방산 오메가3 철분과 칼슘 비타민A 필수지방...
2,치킨사골육수,닭발 닭가슴살 황태 표고버섯 대추 콜라겐 콘드로이틴 피부건강 관절건강 ...
3,스팀제주말고기소시지,말고기 당근 케일 슈퍼푸드 마키베리 표고버섯 아기치즈 팔미톨레산 베타...
4,강아지 보양식 바른 삼계탕,닭발 무항생제 닭안심 소떡심 당근 브로컬리 표고버섯 황태 대추 달걀 ...
5,한입오리,오리안심 단백질 불포화지방산 피부 혈관건강
6,한입치킨,닭안심 단백질 니아신 근육강화 신진대사 면역력
7,무염황태체,황태 아미노산 메티오닌 간기능
8,오리목뼈,오리목뼈 단백질 무기질 콜레스테롤저하 독소배출 스트레스해소 치석제거
9,홍연어저키,홍연어 DHA EPA 오메가3 저지방산 향산화효과 시력개선 심혈관개선 ...


In [55]:
tfid = TfidfVectorizer()
tfid_matrix = tfid.fit_transform(df['BOW'])
cosine_sim = cosine_similarity(tfid_matrix, tfid_matrix)
print(cosine_sim)

[[1.         0.10046301 0.68658542 ... 0.         0.         0.        ]
 [0.10046301 1.         0.16439049 ... 0.         0.         0.        ]
 [0.68658542 0.16439049 1.         ... 0.         0.09663947 0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.09663947 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [56]:
indices = pd.Series(df['상품명'])

In [72]:
def recommend(title, cosine_sim = cosine_sim):
    recommended_movies = []
    idx = indices[indices == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_movies.append(list(df['상품명'])[i])
        
    return recommended_movies

In [73]:
indices[:20]

0             한우사골육수
1            견냥갱(장어)
2             치킨사골육수
3         스팀제주말고기소시지
4     강아지 보양식 바른 삼계탕
5               한입오리
6               한입치킨
7             무염황태체 
8               오리목뼈
9              홍연어저키
10           제주말고기육포
11          견냥갱(소고기)
12            치킨안심육포
13          스팀 오리소시지
14            바르다 화식
15    단호박 연어 파운드 케이크
16             송아지목뼈
17            오리안심육포
18          시금치케일우유껌
19         블루베리비트우유꼄
Name: 상품명, dtype: object

In [74]:
df.head(15)

,상품명,BOW
0,한우사골육수,우족 사골뼈 황태 표고버섯 대추 콜라겐 콘드로이틴 피부건강 관절 기력회복
1,견냥갱(장어),바다장어 황태 한천가루 불포화지방산 오메가3 철분과 칼슘 비타민A 필수지방...
2,치킨사골육수,닭발 닭가슴살 황태 표고버섯 대추 콜라겐 콘드로이틴 피부건강 관절건강 ...
3,스팀제주말고기소시지,말고기 당근 케일 슈퍼푸드 마키베리 표고버섯 아기치즈 팔미톨레산 베타...
4,강아지 보양식 바른 삼계탕,닭발 무항생제 닭안심 소떡심 당근 브로컬리 표고버섯 황태 대추 달걀 ...
5,한입오리,오리안심 단백질 불포화지방산 피부 혈관건강
6,한입치킨,닭안심 단백질 니아신 근육강화 신진대사 면역력
7,무염황태체,황태 아미노산 메티오닌 간기능
8,오리목뼈,오리목뼈 단백질 무기질 콜레스테롤저하 독소배출 스트레스해소 치석제거
9,홍연어저키,홍연어 DHA EPA 오메가3 저지방산 향산화효과 시력개선 심혈관개선 ...


In [75]:
recommend('한입오리')

['오리안심육포',
 '오리황태말이',
 '오리안심육포',
 '스팀 오리소시지',
 '스팀 치킨소시지',
 '오리장각',
 '소떡심',
 '코코넛먼치킨 ',
 '한입치킨',
 '치킨안심육포']

In [77]:
recommend('치킨안심육포')

['한입치킨',
 '스팀 치킨소시지',
 '오리황태말이',
 '캥거루미트볼',
 '코코넛먼치킨 ',
 '소떡심',
 '야채듬뿍테린 ',
 '견냥갱(치킨)',
 '말고기 치즈밀 ',
 '오리안심육포']